In [2]:
hidden_size=256
num_layers = 2
input_size = 28
sequence_length = 28
num_class = 10
learning_rate = 0.001
batch_size = 64
num_eppochs = 3
load_model = True

In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
import torchvision.datasets as dataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
import torchvision.transforms as transforms

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class BNN(nn.Module):
    def __init__(self, input_size , hidden_size , num_layers,  num_class):
        super(BNN , self).__init__()
        self.hidden_size = hidden_size
        self.num_layer = num_layers
        self.lstm = nn.LSTM(input_size,hidden_size , num_layers , batch_first=True , bidirectional=True )
        self.Fc = nn.Linear(hidden_size*2 ,num_class)
    def forward(self , x):
        ho = torch.zeros(self.num_layer*2 ,x.size(0),self.hidden_size).to(device=device)#hidden state or shoert term memory
        co = torch.zeros(self.num_layer*2 ,x.size(0),self.hidden_size).to(device=device)#cell state or long terem memory
        out ,_= self.lstm(x ,(ho , co))#output
        out = self.Fc(out[: , -1 , :])#last timesout
        return out        

In [6]:
class RNN(nn.Module):
    def __init__(self, input_size , hidden_size , num_layers ,num_class):
        super(RNN , self).__init__()
        self.hidden_size = hidden_size
        self.num_layer = num_layers
        self.Rnn = nn.RNN(input_size,hidden_size , num_layers , batch_first=True )
        self.Fc = nn.Linear(hidden_size*sequence_length,num_class)
    def forward(self , x):
        ho = torch.zeros(self.num_layer,x.size(0),self.hidden_size).to(device=device)
        out ,_= self.Rnn(x , ho)
        out = out.reshape(out.shape[0] , -1)
        out = self.Fc(out)
        return out

In [7]:
def save_checkpoint(state , filename = "my_checkpointt.pth.tar"):
    print("=>saving checkpoint")
    torch.save(state , filename)

In [8]:
train_datasets = datasets.MNIST(root ='dataset/' , train= True ,transform = transforms.ToTensor() , download=True)
train_lader = DataLoader(train_datasets , batch_size= batch_size ,shuffle=True)
test_dataset = datasets.MNIST(root ='dataset/' , train= False ,transform = transforms.ToTensor() , download = True)
test_dataloader = DataLoader(test_dataset , batch_size=batch_size , shuffle=True)


In [9]:
model = BNN(input_size,hidden_size,num_layers,num_class).to(device=device)


In [10]:
critision = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [11]:
def load_checkpoint(checkpoint):
    print("=>load checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer dict"])

In [12]:
for epochs in range(num_eppochs):
    losses = []

    if epochs==2:
        check_point = {'state_dict':model.state_dict() , 'optimizer dict': optimizer.state_dict()}
        save_checkpoint(check_point)
    for batch_idx ,(data ,targets) in enumerate(train_lader):
        data = data.to(device = device).squeeze(1)
        targets = targets.to(device = device)
        scores = model(data)
        loss = critision(scores,targets)
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

=>saving checkpoint


In [11]:
def check_accuracy(loader,model):
    if loader.dataset.train:
        print("accuracy for train dataset")
    else:
        print("accuracy fo test datsets")
    num_correct = 0
    num_samples=0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device = device)
            scores = model(x)
            _,predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
        print(f'got {num_correct}/{num_samples} with accuraccy  {float(num_correct)/float(num_samples)*100:.2f}'
        )
    model.train()
    def load_checkpoint(checkpoint):
        print("=>load checkpoint")
        model.load_state_dict(checkpoint["state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer dict"])
    if load_model:
            load_checkpoint(torch.load("my_checkpointt.pth.tar"))

In [12]:
check_accuracy(train_lader , model)

accuracy for train dataset
got 59040/60000 with accuraccy  98.40
=>load checkpoint


In [ ]:
check_accuracy(test_dataloader , model)

accuracy o test datsets
got 9638/10000 with accuraccy  96.38
